In [ ]:
# @title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title API Key for Ngrok
with open("<txt-file-path-in-google-drive>", "r") as f:
    API_KEY = f.read().strip()
    print(API_KEY)

In [ ]:
# @title Install Dependencies
!pip install diffusers transformers accelerate scipy safetensors

In [ ]:
# @title  Load Stable Diffusion Model

from diffusers import StableDiffusionPipeline
import torch

# Load the model from Hugging Face
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")

In [ ]:
# @title FastAPI Server
!pip install fastapi uvicorn python-multipart
from fastapi import FastAPI
import torch
from diffusers import StableDiffusionPipeline
from pydantic import BaseModel
from PIL import Image
import io
import base64
import uvicorn
from threading import Thread

# Initialize API
app = FastAPI()

# Load Stable Diffusion Model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")

# Define request model
class PromptRequest(BaseModel):
    prompt: str

@app.post("/generate")
def generate_image(request: PromptRequest):
    image = pipe(request.prompt).images[0]

    # Convert to base64
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    return {"image": img_str}

# Start API in a background thread
def start_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=start_api, daemon=True).start()


In [ ]:
# @title Get the Public API URL (Using ngrok)
!pip install pyngrok
!ngrok authtoken $API_KEY

from pyngrok import ngrok
public_url = ngrok.connect(8000)
print("Public API URL:", public_url)